# Step One: Setup

### Integrating Environmental Factors into Storm Water Structure Point Layer

**Main Layer**: **Storm Water Structure Point Layer**

#### Environmental Factors:
1. **Distance to Stream**
2. **Distance to Roads**
3. **Digital Elevation Model (DEM)**


### 1. Distance to Streams

###### Source: https://www.nconemap.gov/datasets/e6b5694f81ed49c6a0f3600540f49878_0/explore?location=35.315422%2C-80.715475%2C14.61

In [23]:
streamPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\01_streamDistance\streamClipped.shp"

### 2. Distance to Roads

###### Source: https://connect.ncdot.gov/resources/gis/Pages/GIS-Data-Layers.aspx

In [24]:
roadPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\02_roadsDistance\roadsClipped.shp"

### 3. Mecklenburg DEM

###### Source: https://sdd.nc.gov/DownloadFiles.aspx?path=DEMMosaicsbyCounty/DEM50_CountywideRasters

In [25]:
demPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\03_meckDEM\Mecklenburg_Ground_50ft.tif"

### 4. Storm Water Structures

In [26]:
stormPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\00_theData\inlet.csv"

# Step Two: Coding

### Import Statements

In [37]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from rasterstats import point_query

### Structures CSV into DataFrame

In [28]:
stormpoints_df = pd.read_csv(stormPath)
stormpoints_gdf = gpd.GeoDataFrame(
    stormpoints_df,
    geometry=gpd.points_from_xy(stormpoints_df['Longitude'], stormpoints_df['Latitude']),
    crs="EPSG:2264"
)

### Loading Road GDF

In [29]:
roads_gdf = gpd.read_file(roadPath)

target_crs = "EPSG:2264"
stormpoints_gdf = stormpoints_gdf.to_crs(target_crs)
print(stormpoints_gdf.crs)
roads_gdf = roads_gdf.to_crs(target_crs)
# roads_gdf was originally loaded as 6360 but GeoPandas can't handle Compound CRS
print(roads_gdf.crs)



c:\Users\rcarte64\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\pyogrio\raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(


EPSG:2264
EPSG:2264


### Distance to Nearest Road

In [30]:
def min_distance_to_roads(stormpoint, roads):
    return roads.distance(stormpoint).min()

stormpoints_gdf['nearest_road'] = stormpoints_gdf['geometry'].apply(
    lambda point: min_distance_to_roads(point, roads_gdf)
)

### Loading Streams GDF

In [31]:
streams_gdf = gpd.read_file(streamPath)
streams_gdf = streams_gdf.to_crs(target_crs)

### Distance to Nearest Stream

In [32]:
def min_distance_to_streams(stormpoint,streams):
    return streams.distance(stormpoint).min()

stormpoints_gdf['nearest_stream'] = stormpoints_gdf['geometry'].apply(
    lambda point: min_distance_to_streams(point, streams_gdf)
)
    

### DEM


In [ ]:
with 

# Step Three: Export


### Results to CSV

In [35]:
outputPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\03_outputTesting\roads_streams_Test.csv"

In [36]:
output_df = stormpoints_gdf[['OBJECTID', 'Latitude', 'Longitude', 'nearest_road','nearest_stream']]
output_df.to_csv(outputPath)